# Lecture 13:  Integral Transforms and D/FFT
----

### Sections

* [Introduction](#Introduction)
* [Learning Goals](#Learning-Goals)
* [On Your Own](#On-Your-Own)
    * [The Fourier Transform](#The-Fourier-Transform)
    * [The Discrete Fourier Transform](#The-Discrete-Fourier-Transform)
* [In Class](#In-Class)
    * [DFT with Numpy Functions](#DFT-with-Numpy-Functions)
    * [Interactive Microscopy Demonstration](#Interactive-Microscopy-Demonstration)
* [Homework](#Homework)
* [Summary](#Summary)
* [Looking Ahead](#Looking-Ahead)
* [Reading Assignments and Practice](#Reading-Assignments-and-Practice)

### Introduction
----

I have examined a few texts and web sites that explain the Fourier transform and the Laplace transform.  I've also tried to wrap my head around a more common-language definition of an "Integral Transform" to aid in understanding - but it continues to elude me.  

There is a very matter of fact definition of "Integral Transform" on Mathworld and I've seen it repeated in various places:

$$g(\alpha) = \int_{a}^{b} f(t) K(\alpha,t) dt $$

These descriptions state that any relationship between f and g such as the one above are generically "Integral Transforms" that map the two functions into each other.  

There are discussions on existence, integrability, and inversion that are necessary to raise, but, for us (right now) we will proceed without further consideration.

The diffusion texts that I've consulted resort to the Laplace transform to remove the time dependence from Fick's second law.  The equations are mapped into a different coordinate system that reduces the partial differential equation to an ODE - thereby making it more easily solved.

Another use of an integral transform, the Fourier Transform, is often used to describe diffraction phenomena.  There are additional physics involved with regards to the geometry of the specimen and the interaction of the electron waves in matter, but, at the core of the description of diffraction is the Fourier Transform.

[Top of Page](#Sections)

### Learning Goals
----

* Be comfortable with the idea of an integral transform.
* Study how to implement the Fourier, and Discrete Fourier transforms.
* Be able to conceptually explain how HRTEM works.

[Top of Page](#Sections)

### On Your Own
----

Here are some reminders about special symbols and the result of an integration:

In [ ]:
import sympy as sp
sp.init_printing(use_latex=True)

In [ ]:
# symbols we will need below
x,y,z,t,c = sp.symbols('x y z t c')
omega = sp.symbols('omega', positive=True)

A reminder that $i$ is the square root of negative one and this is how you specify $i$ in `Sympy`.

In [ ]:
sp.I**2

The natural logarithm of $e$ is $1$:

In [ ]:
sp.log(sp.E)

In SymPy there are two ways to deal with integration.  If you would like to represent an unevaluated integral, you can use the `Integral` function.  If you want to compute the integration of an expression you can use the `integrate` function.

In [ ]:
sp.Integral(sp.E**(sp.I*omega*t),t)

In [ ]:
sp.integrate(sp.E**(sp.I*omega*t),t)

Where we assume there is no zero frequency (as we are dividing by $\omega$) - hence the assumption `positive=True` in the symbol definition above.  (Try replacing $\omega$ with "y" and inspect the result of `sp.integrate()`.)

[Top of Page](#Sections)

#### The Fourier Transform

If we extend the notion of a Fourier series (see the previous lecture) to larger and larger domains, the frequency spectrum required to represent the function becomes more finely divided.  Recall the argument in the trigonometric terms in the functions that computed the even and odd terms of the Fourier series:

$$ \frac{n \pi (\omega +c)}{d} $$

n was the order of the amplitude, c the offset, d the domain width.

If we let the domain go to infinity (implying that the function is not periodic) then each frequency component becomes more finely divided as n/d.  This admits an integral rather than a discrete summation.

Some of the surveyed texts will point out that the non-periodic function and its frequency spectrum are related by the Fourier transform defined as:

$$ \hat{f}(\omega) = \sqrt{\frac{1}{2\pi}} \int^{+\infty}_{-\infty} f(t) \exp[-i \omega t] dt $$

This results in a mapping of the function f(t) into frequency space.  Let us plot a function with a value of $1$ in some specified interval.

The real/imaginary, even/odd nature of the function will have an effect on the terms present in the Fourier transform.  For the purposes of materials crystal structures we will be using even and real functions.  We use the `sp.Piecewise` function to generate a "tophat" function for the Fourier transform.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8, 4

p = sp.Piecewise((0,x<-1),(1,x<1),(0,True))
sp.plot(p);

This function is even and real.

We use the definition of the integral transform above to write an integral statement of the Fourier transform of the top-hat function above.  The integral is $1$ between $c$ and $-c$ and zero elsewhere - so we can integrate **just the non-zero part**.  This is integrated as: 

In [ ]:
sp.Integral(1*sp.exp(-sp.I*2*omega*x),(x,-c,c))

Calling explicitly for the integration and assigning the result to `a`:

In [ ]:
a = sp.sqrt(1/(2*sp.pi))*sp.integrate(1*sp.exp(-sp.I*2*omega*x),(x,-c,c))
a

This does not (at first glance) appear to be a real function due to the two exponential terms, but we can use some of the algebraic methods built into `SymPy` to help.  We can ask for this form using sines and cosines with the `rewrite` method.  Furthermore - we can simplify it further with the expand function.  This is done in one line below, but understand that you might need to build this up in pieces the first time you try on your own:

In [ ]:
solution = sp.expand(a.rewrite(sp.sin))
solution

Here we can use the `subs` (substitution) method to set the value of `c`.  I plotted the square of the function since the intensity of a diffracted wave is the square of the amplitude.

In [ ]:
sp.plot(solution.subs(c,1));

The diffracted intensity from a single slit is the square of the resulting fourier transform.

In [ ]:
sp.plot(solution.subs(c,1)**2);

We could perform the same integration over two top-hat functions and plot those results.

In [ ]:
compositeIntegral = sp.sqrt(1/(2*sp.pi))*sp.Integral(1*sp.exp(-sp.I*2*omega*x),(x,1,2)) + \
sp.sqrt(1/(2*sp.pi))*sp.Integral(1*sp.exp(-sp.I*2*omega*x),(x,-2,-1))
compositeIntegral

In [ ]:
om = compositeIntegral.doit()
om

The diffracted intensity from this pair of slits would appear as:

In [ ]:
sp.plot(om.rewrite(sp.sin).expand()**2)

Or we could functionalize our function to explore other parameters:

In [ ]:
def awesomeFunction(d=4.0, w=1.0):
    result = sp.sqrt(1/(2*sp.pi))*sp.Integral(1*sp.exp(-sp.I*2*omega*x),\
                                     (x,-(d+w),-(d-w))) + \
    sp.sqrt(1/(2*sp.pi))*sp.Integral(1*sp.exp(-sp.I*2*omega*x),\
                                     (x,(d-w),(d+w)))
    return result.doit()

In [ ]:
sp.expand(awesomeFunction(10.,2.).rewrite(sp.sin))

etc.

[Top of Page](#Sections)

#### The Discrete Fourier Transform

We'll start by doing this by hand.  The discrete Fourier Transform is defined [here](http://en.wikipedia.org/wiki/Discrete_Fourier_transform).  Other resources such as Numerical Recipes, the Python help files and many other websites give the formula.

Knowing what we know about matrix algebra, the DFT is straightforward to implement in Python/Numpy.  It may not be memory efficient or fast, but, it is easy to implement.

It is often instructive to review other implementations of the DFT to help you learn how these are computed.  I will be modeling this implementation after one found [here](http://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/).  The DFT definitions are (following the notation of the blog post):

Forward DFT:

$$X_k = \sum_{n=0}^{N-1} x_n \cdot e^{-i~2\pi~k~n~/~N}$$

Inverse DFT:

$$x_n = \frac{1}{N}\sum_{k=0}^{N-1} X_k e^{i~2\pi~k~n~/~N}$$

You know - I've read a few other texts and Jake Vanderplas' explanation is really wonderful.  The next cell and the code that follows is taken right from JVP's Jupyter notebook:

----
For simplicity, we'll concern ourself only with the forward transform, as the inverse transform can be implemented in a very similar manner.  Taking a look at the DFT expression above, we see that it is nothing more than a straightforward linear operation: a matrix-vector multiplication of $\vec{x}$,

$$\vec{X} = M \cdot \vec{x}$$

with the matrix $M$ given by

$$M_{kn} = e^{-i~2\pi~k~n~/~N}$$

With this in mind, we can compute the DFT using simple matrix multiplication as follows:

In [ ]:
import numpy as np
def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array x"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

We can use the "all close" function to check if the result from `DFT_slow` and `Numpy` are close:

In [ ]:
x_signal = np.random.random(1024)
np.allclose(DFT_slow(x_signal), np.fft.fft(x_signal))

----
 

I think it would be instructive to symbolically expand the matrix above.  Because you might have missed how the code above (in particular `n*k`) leads to a two dimensional matrix.  It is matrix multiplication.  Switching to `sympy` symbols to expose the details we can do the following:

In [ ]:
import sympy as sp
from sympy import Matrix
import numpy as np
sp.init_printing()

* `x` is the input vector.
* `k` is the wavenumber or frequency.
* `n` is the component of the input vector.

In [ ]:
x = sp.Matrix(sp.symbols('x0:5'))
n = sp.Matrix(sp.symbols('n0:5')).T
k = sp.Matrix(sp.symbols('k0:5'))
N = sp.symbols('N')
M = (-sp.I*2*sp.pi*k*n/N).applyfunc(sp.exp)

In [ ]:
M*x

Now - think about the implications of the matrix multiplication above.  The best way I can explain it is to think of each frequency element projected into each point of the input vector - the matrix links `k` and `n`.  So - the contribution at each point is a sum of each frequency contribution...just like our dot product of functions.

[Top of Page](#Sections)

### In Class
----

#### DFT with Numpy Functions

Let us start by reading the help file for a few new functions that we will be using.

In [ ]:
?np.fft # This gives us information on the conventions used in the return values of the functions.

In [ ]:
?np.fft.fft # This is the main DFT function we will use.

In [ ]:
?np.fft.fftfreq  # This is a helper function to prepare a vector of frequencies.

In [ ]:
?np.arange  # Points in an evenly spaced interval.

This approach is derived from a nice discussion on FFT in Python found here.  (Incidentally, the "Glowing Python" is an excellent blog on Python computation.)

First we will divide up time into `samplingInterval` sized chunks between 0 and 1.  This will aid in getting the x-axis scaled correctly so that frequency can be read directly off the DFT result.  You can take `samplingInterval` in seconds putting samplingRate in Hz.  Notice the approach here - we could have done this all in one line, but, by intelligently naming our variables and exposing the details of our thoughts the code is more readable:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

samplingRate = 150.0
samplingInterval = 1.0/samplingRate
timeVector = np.arange(0, 1, samplingInterval)

# Print out the first few elements so you can see what is going on:
timeVector[0:10:]

Next we decide on the frequency of our signal and create a list to have a signal to work with. 

In [ ]:
signalFrequency = 10.0;
ourSignal = \
np.sin(2*np.pi*signalFrequency*timeVector) + \
0.5*np.sin(2*np.pi*(2*signalFrequency)*timeVector)

We'll plot our function to check that we are getting what we expect.

In [ ]:
fig = plt.figure()

axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
axes.plot(timeVector, ourSignal, 'r')
axes.set_xlabel('Time')
axes.set_ylabel('Signal')
axes.set_title('Our Modest Signal');

In [ ]:
n = ourSignal.size
frequencies = np.fft.fftfreq(n, d=1.0/samplingRate)
spectrum = np.abs(np.fft.fft(ourSignal))

fig = plt.figure()

axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])

axes.scatter(frequencies, spectrum, c='r', marker='s', alpha=0.4)

axes.set_xlabel('Frequency')
axes.set_ylabel('Amplitude')
axes.set_title('Our Amplitude Spectrum');

I'm leaving a few of the implementation details for your discovery.

[Top of Page](#Sections)

#### Interactive Microscopy Demonstration (Advanced Discussion, Optional)
(Developed by C. Carter at MIT, translated to Python by D. Lewis)


`meshgrid` helps us to evaluate functions on a numpy 2D array.  With a little experimentation you'll see the value of this approach.  It will be instructive to make small (e.g. 10x10) arrays and play around with functions on those arrays.  `Numpy` has a lot of methods that will help you solve problems.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from numpy.fft import *

def atomic_func(x,y):
    param = 64.0
    return (1+np.sin(4*(x+y)*2*np.pi/param))*(1+np.sin(2*(x-2*y)*2*np.pi/param))/4

def aperture(X, Y, xoffset, yoffset, size):
    return (X-xoffset)**2+(Y-yoffset)**2 > size**2

Above we create a couple of helper functions.  The `atomic_func` is our periodic function that you can think of as the density of electrons in a solid.  This is an approximation of our "crystal structure" that we are going to use for our diffraction experiment via the Fourier transform.

The `aperture` function returns `True` or `False` if you are outside or inside the aperture (this seems counterintuitive until you see that we set the elements outside equal to zero).  We also use `meshgrid` to create a two dimensional array to use.

In [ ]:
x = np.arange(0.0,256.0,1.0)
y = np.arange(0.0,256.0,1.0)
X,Y = np.meshgrid(x, y)

Z = atomic_func(X,Y)

Above we map the atomic function.

In [ ]:
P = np.zeros(Z.shape,dtype=complex)
K = np.zeros(Z.shape,dtype=complex)

K = fftshift(fft2(Z, norm='ortho'))

P = np.copy(K)
P[np.where(aperture(X, Y, 128, 128, 3) & aperture(X, Y, 120, 96, 3))] = 0

In this cell we create two more `numpy` arrays (there are other ways to do this) that have the same shape as Z. The `P` array we use to hold the processed Fourier spectrum.  The processing uses `numpy`'s Boolean indexing to set values in P equal to zero when they are "outside" the aperture.  When we get to the images below you'll see what is meant.

Because Python passes by reference we need to call for a copy of K so that we can modify one without changing the other.

From this processed spectrum we will create an image.  The K array holds the whole Fourier spectrum.

In [ ]:
Im = fftshift(ifft2(P))

Above we reprocess `P` into the image `Im`.

In [ ]:
fig0 = plt.figure()
axes0 = fig0.add_axes([0.1, 0.1, 0.8, 0.8])
axes0.imshow(Z, origin='lower')

fig1 = plt.figure()
axes1 = fig1.add_axes([0.1, 0.1, 0.8, 0.8]) # left, bottom, width, height (range 0 to 1)
axes1.imshow(abs(K),origin='lower', cmap=plt.get_cmap('pink'))
aperture1 = plt.Circle((128,128),3**2,color='r', fill = False)
aperture2 = plt.Circle((120,96),3**2,color='y', fill = False)
axes1.add_artist(aperture1)
axes1.add_artist(aperture2)

fig2 = plt.figure()
axes2 = fig2.add_axes([0.1, 0.1, 0.8, 0.8])
axes2.imshow(abs(Im)**2, origin='lower')

plt.show()

[Top of Page](#Sections)

### Homework
----

1.  (Optional) Compute the diffraction pattern expected from a double slit experiment.  
1.  Apply the DFT to an image of your choosing.  Select the low frequency part of the DFT and regenerate the image (i.e. take the inverse FFT) from only these selected frequencies.

Hint:

Use a Boolean selection to zero out parts of the frequency spectrum before you convert back.

### Homework Notes
----
Now that we are a few lectures into the course - you are left to set up your own data structures and own methods to solve the problem.  Don't be afraid to delete everything and start over at the early stages of you work.  Keep a pencil and paper handy and sketch ideas before you try and write them in code.  Look for standard solutions to these problems for guidance.

Regarding the second part of the assignment:  Comment on features that are present/absent from the image.  The image can be anything.  You may find it interesting if the image is a pattern.  One possibility is to pull your image from `Lecture-09` since we know those images have structure and patterns in them.

A small snippet of code that will help you read an image would look like this:

```python
from scipy.ndimage import imread
img = imread('./images/pattern2.jpg', mode='L')
```

[Top of Page](#Sections)

### Summary
----

* Integral transforms map one function space into another function space.  You can find books that include tables of Laplace and Fourier transforms.  Many other transforms exist - but the principle is the same.
* The DFT organizes amplitude information in predictable yet non-intuitive ways.  Read the documentation for the functions you use!  
* Integral transforms are a means for reducing the complexity of certain ODEs and PDEs.
* Diffraction and diffusion are two example applications where integral transforms can be employed.

[Top of Page](#Sections)

### Looking Ahead
----

As we move into solving ODEs to understand viscoelastic behavior you may notice that it will be possible to apply the Laplace transformation to simplify the problem.  We won't formally do this in class - but it would be a nice exercise to try it for yourself.  Don't be afraid to try out the new techniques you are learning to other application areas.  This is how you gain confidence!

[Top of Page](#Sections)

### Reading Assignments and Practice
----

* Pam Champness' book on electron diffraction is a (relatively) easy read on diffraction.  You can always have a look at Cullity, Hammond, or any other book on structure and X-ray/electron characterization.
* Practice taking the FFT of signals you construct by hand.  This is a good step when you are debugging a problem.  You should always have a test case available to determine if your work is correct.

[Top of Page](#Sections)